<a href="https://colab.research.google.com/github/space-owner/Tensorflow-2/blob/main/Text%20Classification%20with%20RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Text Classification with an RNN***
This post is **based on the Tensorflow tutorial** for study purposes. [Link](https://www.tensorflow.org/text/tutorials/text_classification_rnn)

***Learning Point:***
- **```Many-to-One Architecture```**
- **```tf.keras.Bidirectional()```**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

print(">>> tf.version =", tf.__version__)

>>> tf.version = 2.6.0


In [ ]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [ ]:
dataset, info = tfds.load(
    'imdb_reviews', with_info=True, as_supervised=True
)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete1JN2TF/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete1JN2TF/imdb_reviews-test.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete1JN2TF/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
for example, label in train_dataset.take(1):
    print(">>> text =", example.numpy())
    print(">>> label =", label.numpy())

>>> text = b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
>>> label = 0


In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_dataset = test_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [ ]:
for example, label in train_dataset.take(1):
    print(">>> text =", example.numpy()[:3])
    print(">>> label =", label.numpy()[:3])

>>> text = [b'I\'m on the opposite end of the previous comment.<br /><br />First of all, I don\'t think this was intended to be a straight sequel to "The Jerk". I mean, it\'s not titled "The Jerk 2"... it\'s "The Jerk, Too", which leads me to believe that while a lot of the character names are the same, it actually revolves around a completely different person.<br /><br />Think about it: Virtually no connection to the previous movie, other than character names; a totally different story; different cast; and the fact that it\'s a partial musical.<br /><br />I say give this movie some credit. It does have plenty of laughs in it.. Mark Blankfield at his prime.'
 b"But, lets face it... it got a few nostalgic sighs out of me.<br /><br />The show is just so consistently great that it is allowed to have a few hiccups. I get a new season, and just power through them like I have 2-days to live. I like the idea of wrapping it up, but it was much more of an end of season episode which would expla

In [ ]:
VOCAB_SIZE = 1000

encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE
)
encoder.adapt(
    train_dataset.map(lambda text, label: text)
)

In [ ]:
vocab = np.array(encoder.get_vocabulary())

print(">>> vocab example =", vocab[:20])

>>> vocab example = ['' '[UNK]' 'the' 'and' 'a' 'of' 'to' 'is' 'in' 'it' 'i' 'this' 'that'
 'br' 'was' 'as' 'for' 'with' 'movie' 'but']


In [ ]:
encoded_example = encoder(example)[:3].numpy()

print(">>> encoded_example =", encoded_example)

>>> encoded_example = [[142  21   2 ...   0   0   0]
 [ 19 599 403 ...   0   0   0]
 [  1   1 908 ...   0   0   0]]


In [ ]:
for n in range(3):
    print("Original: ", example[n].numpy())
    print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
    print()

Original:  b'I\'m on the opposite end of the previous comment.<br /><br />First of all, I don\'t think this was intended to be a straight sequel to "The Jerk". I mean, it\'s not titled "The Jerk 2"... it\'s "The Jerk, Too", which leads me to believe that while a lot of the character names are the same, it actually revolves around a completely different person.<br /><br />Think about it: Virtually no connection to the previous movie, other than character names; a totally different story; different cast; and the fact that it\'s a partial musical.<br /><br />I say give this movie some credit. It does have plenty of laughs in it.. Mark Blankfield at his prime.'
Round-trip:  im on the [UNK] end of the previous [UNK] br first of all i dont think this was [UNK] to be a straight sequel to the [UNK] i mean its not [UNK] the [UNK] 2 its the [UNK] too which leads me to believe that while a lot of the character [UNK] are the same it actually [UNK] around a completely different [UNK] br think about

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()), output_dim=64, mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
print("model.layers =", [print(layer) for layer in model.layers])

model.layers = [None, None, None, None, None]


In [ ]:
print(
    [layer.supports_masking for layer in model.layers]
)

[False, True, True, True, True]


In [ ]:
sample_text = (
    'The movie was cool. The animation and the graphics '
    'were out of this world. I would recommend this movie.'
)
print(">>> np.array([sample_text]) =", np.array([sample_text]))

predictions = model.predict(np.array([sample_text]))

print(">>> predictions =", predictions[0])

>>> np.array([sample_text]) = ['The movie was cool. The animation and the graphics were out of this world. I would recommend this movie.']
>>> predictions = [-0.00798043]


In [ ]:
padding = "the " * 2000

print(">>> np.array([sample_text, padding]) = ", np.array([sample_text, padding]))

predictions = model.predict(np.array([sample_text, padding]))

print(">>> predictions =", predictions[0])

>>> np.array([sample_text, padding]) =  ['The movie was cool. The animation and the graphics were out of this world. I would recommend this movie.'
 'the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the th

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_dataset, 
    epochs=10,
    validation_data=test_dataset,
    validation_steps=30
)

Epoch 1/10
391/391 [==============================] - 108s 253ms/step - loss: 0.6379 - accuracy: 0.5966 - val_loss: 0.5266 - val_accuracy: 0.7708
Epoch 2/10
391/391 [==============================] - 95s 240ms/step - loss: 0.4597 - accuracy: 0.7928 - val_loss: 0.3788 - val_accuracy: 0.8443
Epoch 3/10
391/391 [==============================] - 97s 245ms/step - loss: 0.3685 - accuracy: 0.8377 - val_loss: 0.3350 - val_accuracy: 0.8521
Epoch 4/10
391/391 [==============================] - 96s 243ms/step - loss: 0.3352 - accuracy: 0.8531 - val_loss: 0.3245 - val_accuracy: 0.8583
Epoch 5/10
391/391 [==============================] - 97s 245ms/step - loss: 0.3187 - accuracy: 0.8623 - val_loss: 0.3171 - val_accuracy: 0.8729
Epoch 6/10
391/391 [==============================] - 93s 235ms/step - loss: 0.3130 - accuracy: 0.8653 - val_loss: 0.3245 - val_accuracy: 0.8464
Epoch 7/10
391/391 [==============================] - 94s 237ms/step - loss: 0.3061 - accuracy: 0.8680 - val_loss: 0.2952 - val_a

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('test_loss:', test_loss)
print('test_ac:', test_acc)

391/391 [==============================] - 48s 120ms/step - loss: 0.3231 - accuracy: 0.8617
test_loss: 0.3231208324432373
test_ac: 0.8616799712181091


### ***Stack Two or More LSTM Layers***

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset,
    validation_steps=30
)

Epoch 1/10
391/391 [==============================] - 195s 455ms/step - loss: 0.6099 - accuracy: 0.6096 - val_loss: 0.4165 - val_accuracy: 0.8188
Epoch 2/10
391/391 [==============================] - 170s 433ms/step - loss: 0.3996 - accuracy: 0.8282 - val_loss: 0.3567 - val_accuracy: 0.8448
Epoch 3/10
391/391 [==============================] - 171s 434ms/step - loss: 0.3447 - accuracy: 0.8562 - val_loss: 0.3325 - val_accuracy: 0.8531
Epoch 4/10
391/391 [==============================] - 170s 433ms/step - loss: 0.3242 - accuracy: 0.8638 - val_loss: 0.3102 - val_accuracy: 0.8776
Epoch 5/10
391/391 [==============================] - 171s 435ms/step - loss: 0.3155 - accuracy: 0.8682 - val_loss: 0.3170 - val_accuracy: 0.8568
Epoch 6/10
391/391 [==============================] - 173s 440ms/step - loss: 0.3137 - accuracy: 0.8674 - val_loss: 0.3084 - val_accuracy: 0.8625
Epoch 7/10
391/391 [==============================] - 169s 428ms/step - loss: 0.3066 - accuracy: 0.8726 - val_loss: 0.3185 -

In [ ]:
sample_text = (
    'The movie was not good. The animation and the graphics '
    'were terrible. I would not recommend this movie.'
)
predictions = model.predict(np.array([sample_text]))
print(">>> predictions = ", predictions)

>>> predictions =  [[-1.5732746]]


In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('test_loss:', test_loss)
print('test_ac:', test_acc)

391/391 [==============================] - 83s 210ms/step - loss: 0.3151 - accuracy: 0.8546
test_loss: 0.3151448965072632
test_ac: 0.854640007019043
